In [1]:
import rawgpy
import time
import json
import urllib.request
from nt import listdir
import os
import re
from os import path
import pandas as pd
import numpy as np
import nltk
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier
# Performance metric
from sklearn.metrics import f1_score


pd.set_option('display.max_colwidth', 300)
meta = pd.read_csv("C:/Users/Matilde/Desktop/GitHub/datamin/genrePrediction/gameClean.csv", sep = ',' , header = None)

meta.columns = ["id","name","description","genres"]
meta.head()

,id,name,description,genres
0,id,name,descr,genres
1,arth,seven day determin fate human begin high tech tokyo rubbl vibrant new york ground halt citi light pari envelop dark despair join mankind struggl massiv invad converg world major citi battl set actual area modern metropolitan citi take flight one futurist aircraft help repel humong enemi stage co...,['Shooter'],NaN
2,10235,0 day,freedom movement flight touch pad teleport move citi hunt enemi grow assort weapon home made spear assault rifl sneak arm target catch guard maneuv full speed around bullet bat head distanc take back control citi terrorist infect help shape game give feedback work make game want encourag read ea...,"['Action', 'Adventure', 'Strategy']"
3,386925,0 hour meltdown,built hour game jam power plant meltdown turn everyth tri stop meltdown timer,['Puzzle']
4,379914,0 robert blaster391 arnold,puzzl platform start control vision made hour ludum dare game jam theme start noth,['Puzzle']


In [2]:
games = pd.DataFrame(meta)
games.head()

,id,name,description,genres
0,id,name,descr,genres
1,arth,seven day determin fate human begin high tech tokyo rubbl vibrant new york ground halt citi light pari envelop dark despair join mankind struggl massiv invad converg world major citi battl set actual area modern metropolitan citi take flight one futurist aircraft help repel humong enemi stage co...,['Shooter'],NaN
2,10235,0 day,freedom movement flight touch pad teleport move citi hunt enemi grow assort weapon home made spear assault rifl sneak arm target catch guard maneuv full speed around bullet bat head distanc take back control citi terrorist infect help shape game give feedback work make game want encourag read ea...,"['Action', 'Adventure', 'Strategy']"
3,386925,0 hour meltdown,built hour game jam power plant meltdown turn everyth tri stop meltdown timer,['Puzzle']
4,379914,0 robert blaster391 arnold,puzzl platform start control vision made hour ludum dare game jam theme start noth,['Puzzle']


In [3]:
genres = [] 
genres1 = [] 
s1= ['']
# extract genres
for i in games['genres']: 
    a = list(i.replace("]",'').replace('[','').replace("'",'').replace(' ','').split(","))
    if a == s1:
        genres.append(' ') 
    else:
        genres.append(a)
        genres1.append(a)

# add to 'movies' dataframe  
games['genre_new'] = genres
games.shape

AttributeError: 'float' object has no attribute 'replace'

In [0]:
games_new =  games[~(games['genre_new'] == ' ')]
games_new.head(1000)

In [0]:
games_new.shape, games.shape

In [0]:
all_genres = sum(genres1,[])


print(len(set(all_genres)))

In [0]:
all_genres = nltk.FreqDist(all_genres) 

# create dataframe
all_genres_df = pd.DataFrame({'Genre': list(all_genres.keys()), 
                              'Count': list(all_genres.values())})
print(list(all_genres.keys()))

In [0]:
g = all_genres_df.nlargest(columns="Count", n = 50) 
plt.figure(figsize=(12,15)) 
ax = sns.barplot(data=g, x= "Count", y = "Genre") 
ax.set(ylabel = 'Count') 
plt.show()

In [0]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(games_new['genre_new'])

# transform target variable
y = multilabel_binarizer.transform(games_new['genre_new'])

In [0]:

f = pd.DataFrame(games_new)
i=0
for genre in list(all_genres.keys()):
    genreli = y[:,i]
    f[genre] = genreli 
    i=i+1
    
print(f)


In [0]:
print(y)

In [0]:
# split dataset into training and validation set
xtrain, xval, ytrain, yval = train_test_split(games_new['description'], y, test_size=0.2, random_state=9)

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [0]:
print(xtrain[1])

In [0]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)
print(xtrain_tfidf[1])

In [0]:
# get the first vector out (for the first document)
first_vector_tfidfvectorizer=xtrain_tfidf[1]

# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False).head(20)


In [0]:
print(xtrain_tfidf)

In [0]:
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

In [0]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [0]:
# fit model on train data
clf.fit(xtrain_tfidf, ytrain)

In [0]:
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(yval, y_pred)

In [0]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [0]:
y_pred[3]

In [0]:
multilabel_binarizer.inverse_transform(y_pred)[3]

In [0]:
f1_score(yval, y_pred, average="micro")

In [0]:
# predict probabilities
y_pred_prob = clf.predict_proba(xval_tfidf)

In [0]:
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

In [0]:
# evaluate performance
f1_score(yval, y_pred_new, average="micro")





In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(yval, y_pred)